Attemping to infer the parameters as spatial-temporal functions from low-fidelity PDE solution (KDE).

The PDE is assumed to have form:
$$
    u_t + \frac{\partial}{\partial x}\bigg[\mathcal{V}(t,x)u\bigg] = \frac{\partial}{\partial x}\bigg[\mathcal{D}(t,x)u_x\bigg]
$$ where $\mathcal{V}, \mathcal{D}$ are parameterized as separate NNs.

(06/15/2023) Method does not work well at capturing time intervals where the PDE solution changes rapidly. The method may capture the steady state solution well.

In [1]:
from PINN.PhysicsInformedROPDF import *
# Testing
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import numpy as np
import scipy

import time
import pylab as pl
from IPython import display
from IPython.display import clear_output

In [ ]:
# modify dataset
full_data_path = "../data/LinearOscillator/OU_Noise_energy.mat"
data = scipy.io.loadmat(full_data_path)
# subsample factor
space_factor = 5
time_factor = 5
new_pmc = (data["v_density"].T)[0:-1:time_factor, 0:-1:space_factor]
new_xgrid = data["xi"][:, 0:-1:space_factor]
new_tgrid = data["tspan"][:, 0:-1:time_factor]
# save new data
new_data_path = "../data/LinearOscillator/OU_Noise_energy_subsample{}.mat".format(int(space_factor*time_factor))
scipy.io.savemat(
    new_data_path, {"pmc": new_pmc, "xgrid": new_xgrid, "tgrid": new_tgrid}
)

In [ ]:
# test saved data
new_data_path = "../data/LinearOscillator/OU_Noise_energy_subsample25.mat"
data = scipy.io.loadmat(new_data_path)
print(data["pmc"].shape, data["xgrid"].shape, data["tgrid"].shape)

Experiments below

In [ ]:
# set random seeds
np.random.seed(10)
torch.manual_seed(10);

data_path = "../data/LinearOscillator/OU_Noise_energy_subsample25.mat"
# create PINN
pinn = PhysicsInformedROPDF(indim=2, outdim=1, data_path=data_path, scheduler="ExponentialLR")

# learn on log-scale
log_scale = True
if log_scale:
    # modify data to be max(log(p), eps)
    pinn.pmc = torch.clamp(torch.log(pinn.pmc), -100)

In [ ]:
for i in range(pinn.nt):
    plt.figure(1);
    plt.plot(pinn.raw_data["pmc"][i, :], color="red")
    plt.ylim([0, 0.04])
    display.clear_output(wait=True)
    display.display(pl.gcf())
    plt.clf()
    time.sleep(0.01)

In [ ]:
# testing training
info = train(
    pinn, pinn.optimizer, pinn.scheduler, batch_size=2**10,
    epochs=1000, batch_print=200, mode="data_only"
)

In [ ]:
# visualize losses
plt.figure(1, figsize=(16, 8));
plt.plot(info["pde_loss"], lw=2, color="red");
plt.title("PDE Loss vs. Epoch");

plt.figure(2, figsize=(16, 8));
plt.plot(info["data_loss"], lw=2, color="blue");
plt.title("Data Loss vs. Epoch");

In [ ]:
query = cartesian_data(pinn.tgrid, pinn.xgrid)

D_pred = pinn.D_nn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()
G_pred = pinn.G_nn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()
p_pred = pinn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()
p_exact = np.log(pinn.raw_data["pmc"])
p_exact = np.clip(p_exact, a_min=-100, a_max=float("inf"))

# grids
tgrid = pinn.tgrid.detach().numpy()
xgrid = pinn.xgrid.detach().numpy()

In [ ]:
plt.figure(1);
plt.contourf(xgrid, tgrid, G_pred);
plt.colorbar()
plt.figure(2);
plt.plot(xgrid, G_pred[0:-1:10, :].T, alpha=0.4, lw=1.5);

In [ ]:
plt.contourf(pinn.xgrid.detach().numpy(), pinn.tgrid.detach().numpy(), p_pred)

In [ ]:
plt.figure(1);
idx = 29
plt.plot(xgrid, np.exp(p_pred[idx, :]), label="predict", color='blue')
plt.plot(xgrid, np.exp(p_exact[idx, :]), label="exact", color="red");
plt.legend();

plt.figure(2);
plt.plot(xgrid, p_pred[idx, :], label="predict", color="blue")
plt.plot(xgrid, p_exact[idx, :], label="exact", color="red");
plt.legend();


In [ ]:
plt.contourf(pinn.xgrid.detach().numpy(), pinn.tgrid.detach().numpy(), p_exact)

In [ ]:
xgrid = pinn.xgrid.detach().numpy()
dx = xgrid[1]-xgrid[0]
plt.figure(1);
fig, ax = plt.subplots(2, 2);
ax[0, 0].plot(xgrid, p_exact[0, :], label="KDE", lw=1.0, color="red");
ax[0, 0].plot(xgrid, p_pred[0, :], label="PINN", lw=1.0, color="blue");
ax[0, 0].set_title(r"$t = 0$");

ax[0, 1].plot(xgrid, p_exact[125, :], label="KDE", lw=1.0, color="red");
ax[0, 1].plot(xgrid, p_pred[125, :], label="PINN", lw=1.0, color="blue");
ax[0, 1].set_title(r"$t = 2.5$");

ax[1, 0].plot(xgrid, p_exact[250, :], label="KDE", lw=1.0, color="red");
ax[1, 0].plot(xgrid, p_pred[250, :], label="PINN", lw=1.0, color="blue");
ax[1, 0].set_title(r"$t = 5.0$");

ax[1, 1].plot(xgrid, p_exact[-1, :], label="KDE", lw=1.0, color="red");
ax[1, 1].plot(xgrid, p_pred[-1, :], label="PINN", lw=1.0, color="blue");
ax[1, 1].set_title(r"$t = 10.0$");

fig.set_figwidth(8);
fig.tight_layout(pad=0.5);

In [ ]:
plt.figure(2);
# relative error in L^2 from KDE benchmark
l2_rel_err = np.sum(((np.exp(p_pred)-np.exp(p_exact))**2)*dx, 1) / np.sum((np.exp(p_exact)**2)*dx, 1)
tgrid = pinn.tgrid.detach().numpy()
plt.plot(tgrid, l2_rel_err, color="red", lw=1.5);
plt.grid(True);
plt.title(r"Relative error in $L^2$");
plt.xlabel(r"$t$"); 
plt.ylabel(r"$L^2(\mathbb{R})$ error");